<a href="https://colab.research.google.com/github/projectgroup2110/Stock-Sentiment-Analysis-NLP-project/blob/master/Copy_of_Con_Anime_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone -l -s https://github.com/projectgroup2110/BEProjectGanAnime.git cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [0]:
pwd

'/content'

In [0]:
cd /content/cloned-repo/src

/content/cloned-repo/src


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# file_id = '19SaalG6mDQwiBFMVApcap58ob-t3mznU'
# downloaded = drive.CreateFile({'id': file_id})


In [0]:
cp /content/drive/My\ Drive/datasets/ACGAN_generator.ckpt /content/cloned-repo/src/mymodel/

In [0]:
!python3 test.py -t fix_hair_eye --hair purple --eye red -d /content/cloned-repo/src/mymodel/

In [0]:
!pip install anvil-uplink

     |████████████████████████████████| 61kB 4.9MB/s 
  Created wheel for ws4py: filename=ws4py-0.3.4-cp36-none-any.whl size=41809 sha256=4375df96d5a4048b97275595b15100856cdb7ff09581ca105bbd112f1c6df51b
  Stored in directory: /root/.cache/pip/wheels/19/1f/0d/beff5822af761b66067b5e0b251a9c66af3ae15828ee9a8f15
Successfully built ws4py


In [0]:
import anvil.server
anvil.server.connect('QHZWYIO4L5CHOLFY4XR6TS7D-OYWK4Q5T4YWKQLN2')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [0]:
import torch
import torch.nn
import torch.optim as optim
import torchvision.transforms as Transform
from torchvision.utils import save_image

import numpy as np
import os

import datasets
import ACGAN
import utils

import folium
from folium import plugins
from PIL import Image

import anvil.media

# from argparse import ArgumentParser

hair_mapping =  ['orange', 'white', 'aqua', 'gray', 'green', 'red', 'purple', 
                 'pink', 'blue', 'black', 'brown', 'blonde']
hair_dict = {
    'orange' : 0,
    'white': 1, 
    'aqua': 2,
    'gray': 3,
    'green': 4,
    'red': 5,
    'purple': 6,
    'pink': 7,
    'blue': 8,
    'black': 9,
    'brown': 10,
    'blonde': 11
}

eye_mapping = ['black', 'orange', 'pink', 'yellow', 'aqua', 'purple', 'green', 
               'brown', 'red', 'blue']
eye_dict = {
    'black': 0,
    'orange': 1,
    'pink': 2,
    'yellow': 3,
    'aqua' : 4,
    'purple': 5,
    'green': 6,
    'brown': 7,
    'red': 8,
    'blue': 9
}


# parser = ArgumentParser()
# parser.add_argument('-t', '--type', help = 'Type of anime generation.', 
#                     choices = ['fix_noise', 'fix_hair_eye', 'change_hair', 'change_eye', 'interpolate'], 
#                     default = 'fix_noise', type = str)
# parser.add_argument('--hair', help = 'Determine the hair color of the anime characters.', 
#                     default = None, choices = hair_mapping, type = str)
# parser.add_argument('--eye',  help = 'Determine the eye color of the anime characters.',
#                     default = None, choices = eye_mapping, type = str)
# parser.add_argument('-s', '--sample_dir', help = 'Folder to save the generated samples.',
#                     default = '../generated', type = str)
# parser.add_argument('-d', '--model_dir', help = 'Folder where the trained model is saved',
#                     default = '../models', type = str)
# args = parser.parse_args()

def generate_by_attributes(model, device, latent_dim, hair_classes, eye_classes, hair_color, eye_color):
    hair_tag = torch.zeros(64, hair_classes).to(device)
    eye_tag = torch.zeros(64, eye_classes).to(device)
    hair_class = hair_dict[hair_color]
    eye_class = eye_dict[eye_color]
    for i in range(64):
        hair_tag[i][hair_class], eye_tag[i][eye_class] = 1, 1
    
    tag = torch.cat((hair_tag, eye_tag), 1)
    z = torch.randn(64, latent_dim).to(device)
    
    output = model(z, tag)
    #save_image(utils.denorm(output), '{}/{} hair {} eyes.png'.format(args.sample_dir, hair_mapping[hair_class], eye_mapping[eye_class]))
    save_image(utils.denorm(output), '{}/{} hair {} eyes.png'.format('../generated', hair_mapping[hair_class], eye_mapping[eye_class]))
    ##save_image(utils.denorm(output), '{}/{} hair {} eyes.jpg'.format('../generated', hair_mapping[hair_class], eye_mapping[eye_class]))

    #utils.denorm(output)

    #return output








@anvil.server.callable   
def main(hair,eye):
    # if not os.path.exists(args.sample_dir):
    #     os.mkdir(args.sample_dir)
    if not os.path.exists('../generated'):
        os.mkdir('../generated')
    latent_dim = 100
    hair_classes = 12
    eye_classes = 10
    batch_size = 1

    device = 'cpu'
   # G_path = '{}/ACGAN_generator.ckpt'.format(args.model_dir)
    G_path = '/content/cloned-repo/src/mymodel/ACGAN_generator.ckpt'

    G = G = ACGAN.Generator(latent_dim = latent_dim, class_dim = hair_classes + eye_classes)
    prev_state = torch.load(G_path)
    G.load_state_dict(prev_state['model'])
    G = G.eval()

    # if args.type == 'fix_hair_eye':
    #     generate_by_attributes(G, device, latent_dim, hair_classes, eye_classes, args.hair,  args.eye)
    # elif args.type == 'change_eye':
    #     eye_grad(G, device, latent_dim, hair_classes, eye_classes)
    # elif args.type == 'change_hair':
    #     hair_grad(G, device, latent_dim, hair_classes, eye_classes)
    # elif args.type == 'interpolate':
    #     interpolate(G, device, latent_dim, hair_classes, eye_classes)
    # else:
    #     fix_noise(G, device, latent_dim, hair_classes, eye_classes)

    generate_by_attributes(G, device, latent_dim, hair_classes, eye_classes, hair,  eye)

    # result_image = Image.open('/content/cloned-repo/generated/{} hair {} eyes.png'.format(hair,eye))  
    # return result_image
    return(anvil.media.from_file('/content/cloned-repo/generated/{} hair {} eyes.png'.format(hair,eye)))
    
# if __name__ == "__main__":
#     main()